In [1]:
#give google drive accuss to your drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install rasterio #need to install before importing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 40.4 MB/s eta 0:00:00


In [4]:
#import needed environments
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from rasterio.mask import mask

#create a function that has the inputs as the raster file and of the shapefile path
def zonal_statistics(raster_path, shapefile_path):

    # Create a with loop that will load the raster data
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)  # reads the first band
        nodata_value = src.nodata  # gets the NoData value

    # loads the shapefile
    shp = gpd.read_file(shapefile_path)

    # if statement checks to see if shapefile has an 'id' column
    if 'id' not in shp.columns:
        raise ValueError("The shapefile needs a column named 'id'")

    # create a variable to store stats/results
    stats = []

    # loops through each polygon in the shapefile
    for _, row in shp.iterrows():
        polygon = [row['geometry']]  # gets the polygon geometry

        # with statment that masks the raster using the polygon
        with rasterio.open(raster_path) as src:
            out_image, out_transform = mask(src, polygon, crop=True)

        # flattens the array to calculate mean and std
        out_image = out_image.flatten()  # Flatten the array

        # gets rid of any NoData values
        if nodata_value is not None:
            out_image = out_image[out_image != nodata_value]

        # this if else statement calculates the statistics
        if len(out_image) > 0:  # this only calculates the stats for valid pixels
            mean_value = np.mean(out_image)
            std_value = np.std(out_image)
        else:
            mean_value = np.nan  # If there is no valid data it will return NaN
            std_value = np.nan

        # appends data
        stats.append({
            'id': row['id'],  # Unique ID from shapefile
            'mean': mean_value,
            'std': std_value
        })

    # converts the results into a DataFrame
    result_df = pd.DataFrame(stats)

    return result_df


In [5]:
# calling code
raster_path = '/content/drive/MyDrive/GIS 512/data for lab/lab-6-task-data/ndvi.tif'
shapefile_path = '/content/drive/MyDrive/GIS 512/data for lab/lab-6-task-data/grid.geojson'
df = zonal_statistics(raster_path, shapefile_path)
print(df)

      id      mean       std
0     41  0.405855  0.067818
1     42  0.406319  0.058588
2     61  0.424627  0.065736
3     62  0.417399  0.071759
4     63  0.363553  0.140118
..   ...       ...       ...
367  560  0.333018  0.111100
368  561  0.334814  0.114300
369  562  0.301963  0.115530
370  581  0.312016  0.097210
371  582  0.294459  0.103302

[372 rows x 3 columns]
